#### Idea

- Access spotify with authorization. Get one(all) of your playlists and tracks. 
- Get information/features of those tacks and cluster them together to find trends in your taste of music.
- Maybe sort by dates and see how your taste has evolved?
- ^I like this idea

---Damn I think I am taking my music too seriously

##### Roadmap
We don't have time, so lets get it working quickly. Following are the steps
- Understand how to interact with spotify api
- register your app
- get access to your playlists. Get songs from a playlist. This should conclude phase 1.
- get features for your songs. Save them in database? Plot them?
- Advanced: 
  - Run a clustering algorithm to find labels to define your music
  - Try to find related artists, go through their music and find similar?

In [5]:
import spotipy
sp = spotipy.Spotify()

results = sp.search(q='beethoven', limit=20)
for i, t in enumerate(results['tracks']['items']):
    print (' ', i, t['name'])

  0 Sonata No. 14 "Moonlight" in C-Sharp Minor", Op. 27 No. 2: I. Adagio sostenuto
  1 Piano Sonata No. 8 in C Minor, Op. 13 'Pathétique': II. Adagio cantabile
  2 25 Bagatelles, WoO. 59: No. 25 in A Minor 'Für Elise'
  3 Piano Sonata No. 23 in F Minor, Op. 57 'Appassionata': I Allegro assai
  4 Piano Sonata No. 23 in F Minor, Op. 57 'Appassionata': II Andante con moto
  5 Piano Sonata No. 14 in C-Sharp Minor, Op. 27 No. 2 'Moonlight': I. Adagio sostenuto
  6 Piano Sonata No. 23 in F Minor, Op. 57 'Appassionata': III Allegro, ma non troppo - Presto
  7 Six Bagatelles, Op. 126: V. Quasi allegretto
  8 Piano Sonata No. 26 in E-Flat Major, Op. 81a 'Les Adieux': I. Adagio - Allegro "Les Adieux"
  9 Piano Sonata No.12 in A-Flat Major, Op. 26: Marcia funèbre sulla morte d'un Eroe - Maestoso andante
  10 Piano Sonata No. 23 in F Minor, Op. 57 'Appassionata': I. Allegro assai
  11 6 Minuets, WoO. 10: No. 2 in G Major
  12 Piano Sonata No. 21 in C Major, Op. 53 "Waldstein": I. Allegro con brio


In [28]:

from flask import Flask, request, render_template
from flask import Flask, request, session, g, redirect, url_for, abort, \
     render_template, flash
import sqlite3
from werkzeug.utils import secure_filename
import os
import requests
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json


In [29]:
#coldpaly - spotify:artist:4gzpq5DPGxSnKTe4SA8HAU
response = requests.get("https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU")
print (response)

<Response [200]>


In [36]:
jsonstr = response.content.decode()
jsonstr

'{\n  "external_urls" : {\n    "spotify" : "https://open.spotify.com/artist/4gzpq5DPGxSnKTe4SA8HAU"\n  },\n  "followers" : {\n    "href" : null,\n    "total" : 6301926\n  },\n  "genres" : [ "permanent wave", "pop", "pop christmas", "rock" ],\n  "href" : "https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU",\n  "id" : "4gzpq5DPGxSnKTe4SA8HAU",\n  "images" : [ {\n    "height" : 1000,\n    "url" : "https://i.scdn.co/image/31fa9aa639d539dc93ab9e3570c7c70a1b5c64f8",\n    "width" : 1000\n  }, {\n    "height" : 640,\n    "url" : "https://i.scdn.co/image/f700aaf6ebf1f0105d4eece756d924ab7efc3441",\n    "width" : 640\n  }, {\n    "height" : 200,\n    "url" : "https://i.scdn.co/image/32e13d2c37af4817d1cbf09742c1f51a328820c7",\n    "width" : 200\n  }, {\n    "height" : 64,\n    "url" : "https://i.scdn.co/image/3f31f2b7e890c2e59a9e4d67ea37f459e7036991",\n    "width" : 64\n  } ],\n  "name" : "Coldplay",\n  "popularity" : 91,\n  "type" : "artist",\n  "uri" : "spotify:artist:4gzpq5DPGxSnKTe4SA8H

In [37]:
jsloads = json.loads(jsonstr)

In [39]:
jsloads['name']

'Coldplay'

In [60]:
#coldpaly - spotify:artist:4gzpq5DPGxSnKTe4SA8HAU
# response = requests.get("https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU")
response = requests.get("https://api.spotify.com/v1/search?q='coldplay'&type=artist")
print (response)
jsonstr = response.content.decode()
jsloads = json.loads(jsonstr)


<Response [200]>


In [61]:
jsloads['artists'].keys()

dict_keys(['limit', 'offset', 'href', 'items', 'total', 'next', 'previous'])

In [63]:
jsloads['artists']['items'][2]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/58naUc4Fd9OoP6UrdrpgFO'},
 'followers': {'href': None, 'total': 72},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/58naUc4Fd9OoP6UrdrpgFO',
 'id': '58naUc4Fd9OoP6UrdrpgFO',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/7e544bb9b858905f95ed9aa96a186ceeca94f282',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/072dd1a6fcf580629efa78f55c4e76df6a509780',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/03ca05365ef52b26f844a359a27cd0f59cadd00c',
   'width': 64}],
 'name': 'ColdPlay Wu',
 'popularity': 0,
 'type': 'artist',
 'uri': 'spotify:artist:58naUc4Fd9OoP6UrdrpgFO'}